# Load libraries

In [9]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
from glob import glob
import uproot
import keras


In [2]:
import os
print(os.environ['LD_LIBRARY_PATH'])
print('and\n', os.environ['PATH'])

KeyError: 'LD_LIBRARY_PATH'

In [9]:
# Our package from cern gitlab
from nnlar.datashaper import DataShaper

ModuleNotFoundError: No module named 'nnlar'

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

print("Available devices:", gpus)
# limit VRAM usage if GPU is available
if len(gpus) > 0:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*0.5)])

#tensorflow.config.gpu.set_per_process_memory_fraction(0.75)
#tensorflow.config.gpu.set_per_process_memory_growth(True)
#tensorflow.GPUOptions(per_process_gpu_memory_fraction=0.33)
from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Flatten, LSTM, GRU, SimpleRNN
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau

from collections import deque
import time
import random
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error as mse


: 

In [ ]:
# Load libraries if they are changed
%load_ext autoreload
%autoreload 2

# Show matplotlib plots without explicitly calling plt.show()
%matplotlib inline

: 

## Load data

In [ ]:
mu140 = "/atlas/laatu/dataset/rdgap_mu140.h5"

ds = DataShaper.from_h5(mu140)
X, X_valid, X_test, y, y_valid, y_test = ds()

print(X.shape)

: 

## Create Keras model

In [5]:
model = Sequential()

model.add(LSTM(10, input_shape=(5, 1), return_sequences=False))
model.add(Dense(1, activation='linear'))


model.compile(loss="mean_squared_error", optimizer=Adam(lr=0.001))

model.summary()



NameError: name 'Sequential' is not defined

In [ ]:
checkpoint_filepath = '/tmp/rnn_checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)




reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.00001, verbose=1)

h=model.fit(X,y,validation_data=(X_valid,y_valid), epochs=1, batch_size=128, shuffle=True, callbacks=[model_checkpoint_callback, reduce_lr])

model.load_weights(checkpoint_filepath)

: 

## Example QKeras model

In [12]:
import tensorflow as tf

import qkeras
from qkeras import *
qkeras.quantizers.set_internal_sigmoid("real")

In [13]:
qrnn_model = Sequential()
seq_len=5

bitwidth = (8,2,0,1)
qrnn_model.add(QLSTM(units=10, input_shape=(seq_len,1), activation="quantized_tanh(10, use_real_tanh=True)",
                        recurrent_activation="quantized_sigmoid(10)",
                        kernel_quantizer=quantized_bits(*bitwidth),
                        state_quantizer=quantized_bits(*bitwidth),
                        recurrent_quantizer=quantized_bits(*bitwidth),
                        bias_quantizer=quantized_bits(*bitwidth)
                ))

qrnn_model.add(QDense(units=1, activation="quantized_relu(10)",
                           kernel_quantizer=quantized_bits(*bitwidth),
                          bias_quantizer=quantized_bits(*bitwidth),
                     ))

qrnn_model.compile(loss='mse', optimizer=Adam(0.0001))
qrnn_model.summary()

NameError: name 'Sequential' is not defined

## Load weights of the Keras model

In [ ]:
qrnn_model.set_weights(model.get_weights())

: 

## Quantization aware training for the QKeras model

In [1]:
checkpoint_filepath = '/tmp/qrnn_checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)




reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.00001, verbose=1)

q_hist=qrnn_model.fit(X,y,validation_data=(X_valid,y_valid), epochs=30, batch_size=20, shuffle=True, callbacks=[model_checkpoint_callback, reduce_lr])

qrnn_model.load_weights(checkpoint_filepath)

NameError: name 'tf' is not defined

: 